<div style="text-align: center; line-height: 0; padding-top: 2px;">
  <img src="https://www.quantiaconsulting.com/logos/quantia_logo_orizz.png" alt="Quantia Consulting" style="width: 600px; height: 250px">
</div>

 # MongoDB 
 
 In this notebook you will see how to use Python to communicate with MongoDB


In [ ]:
from pymongo import MongoClient
import pandas as pd
import qcutils

In [ ]:
HOST = qcutils.read_config_value(key="mongodb.host", cf_path="config/nosql-config.yaml")
PORT = qcutils.read_config_value(key="mongodb.port", cf_path="config/nosql-config.yaml")

MONGO_URL = 'mongodb://{}:{}'.format(HOST, PORT)
DB_NAME = 'quantia'
COLLECTION_NAME = 'tweet'
USER = qcutils.read_config_value(key="mongodb.username", cf_path="config/nosql-config.yaml")
PASSWORD = qcutils.read_config_value(key="mongodb.pwd", cf_path="config/nosql-config.yaml")

## Loading the driver

In [ ]:
client = MongoClient(MONGO_URL, username=USER, password=PASSWORD)
db = client[DB_NAME]

## Read


#### Query by MongoDB id

Select the tweet saved with the given MongoDB id

In [ ]:
from bson.objectid import ObjectId
db[COLLECTION_NAME].find_one({"_id":ObjectId("5a9d26f8b986449b68b6690e")})

### Query by field value

Select all the English tweets

In [ ]:
db[COLLECTION_NAME].find({"lang":"en"})

Mongo returns are cursors over the result set, so we need to parse it to retrieve the data.

In [ ]:
# We put :100 to limit the results
list(db[COLLECTION_NAME].find({"lang":"en"})[:100])

This is very inefficient, as it loads all the data into an array. However, for sake of time we will use the previous way thorough the notebook

The correct way is to iterate over it. 

In the next cell we will print the first **5** tweets returned by the previous query

In [ ]:
for tweet in db[COLLECTION_NAME].find({"lang":"en"})[:5]: 
    print(tweet)

We can also load the data in a dataframe

In [ ]:
result = db[COLLECTION_NAME].find({"lang":"en"})[:10]
pd.DataFrame(result)

#### Query using **comparison** operators

Retrieve the first 100 tweets posted during March 2018

You can use the `[:N]` notation as shortcut to define **limits** in the MongoDB query

In [ ]:
from datetime import datetime

query = {
    "date":{
    "$gte":datetime.strptime("2018-03-01T00:00:00","%Y-%m-%dT%H:%M:%S"),
    "$lt":datetime.strptime("2018-04-01T00:00:00","%Y-%m-%dT%H:%M:%S"),
    }
}

result = db[COLLECTION_NAME].find(query)[:100]
pd.DataFrame(result)

Retrieve the first 100 **non-English** tweets posted during March 2018


In [ ]:
query = {
    "date":{
    "$gte":datetime.strptime("2018-03-01T00:00:00","%Y-%m-%dT%H:%M:%S"),
    "$lt":datetime.strptime("2018-04-01T00:00:00","%Y-%m-%dT%H:%M:%S"),
    },
    "lang":{
        "$ne":"en"
    }
}
result = db[COLLECTION_NAME].find(query)[:100]
pd.DataFrame(result)

Retrieve all the tweets that are **not** originating from the United States

You can query the embedded fields by using the dot notation

In [ ]:
query = {
    "place.country_code":{
        "$ne":"US"
    }
}

result = db[COLLECTION_NAME].find(query)[:100]
pd.DataFrame(result)

Find all the tweets that include at least one link

In [ ]:
query = {
    "entities.urls.1":{
        "$exists":True
    }
}

result = db[COLLECTION_NAME].find(query)[:100]
pd.DataFrame(result)

Mongo allows also to filter the fields you want to retrieve.
Now, let's repeat the previous query but keep only the text of the tweet, the date and the location

In [ ]:
query = {
    "place.country_code":{
        "$ne":"US"
    }
}

fields = {
    "text":1,
    "date":1,
    "place.name":1
}

result = db[COLLECTION_NAME].find(query,fields)[:100]
pd.DataFrame(result)

## Insert

Insert a new document in the database

In [ ]:
# Here give a unique name to your collection
WRITE_COLLECTION_NAME = "my_tweets"

In [ ]:
new_tweet = {
        'created_at': 'Fri Jul 31 15:16:13 +0000 2020',
        'id': 1289218313222905860,
        'id_str': '1289218313222905860',
        'text': 'Nasce FABRE, il Consorzio per la valutazione della sicurezza e il monitoraggio di ponti e viadotti in Italia, volut… https://t.co/BvIs79y61y',
        'truncated': True,
        'entities': {
            'hashtags': [],
            'symbols': [],
            'user_mentions': [],
            'urls': [{
                'url': 'https://t.co/BvIs79y61y',
                'expanded_url': 'https://twitter.com/i/web/status/1289218313222905860',
                'display_url': 'twitter.com/i/web/status/1…',
                'indices': [117, 140]
            }]
        },
        'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
        'in_reply_to_status_id': None,
        'in_reply_to_status_id_str': None,
        'in_reply_to_user_id': None,
        'in_reply_to_user_id_str': None,
        'in_reply_to_screen_name': None,
        'geo': None,
        'coordinates': None,
        'place': None,
        'contributors': None,
        'is_quote_status': False,
        'retweet_count': 4,
        'favorite_count': 26,
        'favorited': False,
        'retweeted': False,
        'possibly_sensitive': False,
        'lang': 'it'
    }



In [ ]:
insert_result = db[WRITE_COLLECTION_NAME].insert_one(new_tweet)

Retrieve the **Id** of the inserted document

In [ ]:
insert_result.inserted_id

Mongo does not care about the schema. So you can insert incomplete tweets.

In [ ]:
incomplete_tweet = {
    "text":"Dummy tweet",
    'created_at': 'Fri Jul 31 15:13:13 +0000 2020',
    'id': 1289218313222905860,
    "truncated":False,
    'lang': 'it'
}

In [ ]:
db[WRITE_COLLECTION_NAME].insert_one(incomplete_tweet)

## Update

An update is mongo is composed by a query - as in the find operator - and the actual update. The query is used to select which documents you want to modify, like a WHERE clause in SQL.

Add a boolean field **spam** and set it to true if a tweet text is not in english, is shorter than 20 characters and it wasn't truncated (i.e., the actual text is longer)

In [ ]:
query = {
    "$expr": { "$lte": [ { "$strLenCP": "$text" }, 20 ] },
    "truncated":False,
    "lang":{
        "$ne":"en"
    }
}

update = {
    "$set":{
        "spam":True
    }
}

results = db[WRITE_COLLECTION_NAME].update_many(query,update)

print(results.matched_count, "tweets matched the query")
print(results.modified_count, "tweets were modified")

In [ ]:
query = {
    "spam":True
}

result = db[WRITE_COLLECTION_NAME].find(query)[:100]
pd.DataFrame(result)

## Delete

The delete expects as input a query with the same format as a find()

Delete all the tweets with the information about the **user_id** missing

In [ ]:
query = {
     "user_id":{
         "$exists":False
     }
}

result = db[WRITE_COLLECTION_NAME].delete_many(query)

result.deleted_count

## Aggregate Functions

Calculate the top 10 tweeting users

In [ ]:
pipeline = [{
    "$group":{
        "_id":"$user_id",
        "tweet_number":{
            "$sum":1
        }
    }
},{
    "$sort":{
        "tweet_number":-1
    }
},{
    "$limit":10
}]

result = db[COLLECTION_NAME].aggregate(pipeline)
pd.DataFrame(result)

Find the top 10 most used hashtag

In [ ]:
pipeline = [{
    "$match":{
        "entities.hashtags.1":
        {
            "$exists":True
        }
    }
},{
    "$project":{
        "hashtag":"$entities.hashtags.text"
    }
},{
    "$unwind":"$hashtag"
},{
    "$group":{
        "_id":"$hashtag",
        "count":{
            "$sum":1
        }
    }
},{
    "$sort":{
        "count":-1
    }
},{
    "$limit":10
}]

result = db[COLLECTION_NAME].aggregate(pipeline)
pd.DataFrame(result)

### Lookup (aka Join)

Other information about the users is stored in the **user** collection. In order to combine it with the information in the **tweet** collection we need to join them.

Retrieve where the most active user are located.

In [ ]:
pipeline = [{
    "$group":{
        "_id":"$user_id",
        "tweet_number":{
            "$sum":1
        }
    }
},{
    "$lookup":{
        "from":"user",
        "localField":"_id",
        "foreignField":"_id",
        "as":"popular_locations"
    }
},{
    "$unwind":"$popular_locations"
},{
    "$project":{
        "location":"$popular_locations.location",
        "tweet_number":1
    }
},{
    "$match":{
        "location":{
            "$ne":None
        }
    }
},{
    "$group":{
        "_id":"$location",
        "tweets":{
            "$sum":"$tweet_number"
        },
        "unique_users":{
            "$sum":1
        }
    }
},{
    "$sort":{
        "tweets":-1
    }
}]

result = db[COLLECTION_NAME].aggregate(pipeline)
pd.DataFrame(result)

##### ![Quantia Tiny Logo](https://www.quantiaconsulting.com/logos/quantia_logo_tiny.png) 2020 Quantia Consulting, srl. All rights reserved.